In [15]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error
from numpy.linalg import norm
from surprise import Dataset
from surprise import SVD, KNNBasic, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy

In [3]:
# 1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise.
data = Dataset.load_builtin('ml-100k')
df = pd.DataFrame(data.raw_ratings, columns=['user', 'item', 'rating', 'timestamp'])

In [4]:
# 2. Виведіть перші 5 рядків завантаженого датасету.
df.head(5)

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [6]:
# 3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього датасету.
# Завантажуємо дані у форматі Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user', 'item', 'rating']], reader)

# Алгоритми
algo1 = SVD()  # Алгоритм SVD
algo2 = KNNBasic()  # Алгоритм KNN

In [7]:
# 4. Використайте крос-валідацію для підбору оптимальних параметрів для обох алгоритмів.
cross_validate(algo1, data, measures=['MAE'], cv=5, verbose=True)
cross_validate(algo2, data, measures=['MAE'], cv=5, verbose=True)

Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7396  0.7388  0.7426  0.7387  0.7333  0.7386  0.0030  
Fit time          0.36    0.35    0.37    0.38    0.35    0.36    0.01    
Test time         0.04    0.07    0.05    0.07    0.05    0.06    0.01    
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7750  0.7712  0.7768  0.7739  0.7689  0.7732  0.0028  
Fit time          0.13    0.10    0.15    0.10    0.10    0.11    0.02    
Test time

{'test_mae': array([0.77496068, 0.77124317, 0.77682118, 0.77394969, 0.76889074]),
 'fit_time': (0.12665104866027832,
  0.10031795501708984,
  0.15174007415771484,
  0.09960389137268066,
  0.09664320945739746),
 'test_time': (1.2696731090545654,
  1.2147419452667236,
  1.1789371967315674,
  1.216310977935791,
  1.1611559391021729)}

In [10]:
# 5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки (MAE).
# Тренування обраного алгоритму на всьому датасеті
trainset = data.build_full_trainset()
algo1.fit(trainset)

# Генерація рекомендацій для конкретного користувача
user_id = '186'
user_inner_id = trainset.to_inner_uid(user_id)

# Отримуємо список всіх фільмів
all_items = trainset.all_items()
recommendations = []

for item in all_items:
    # Отримуємо прогнозовану оцінку
    est_rating = algo1.predict(user_id, trainset.to_raw_iid(item)).est
    recommendations.append((trainset.to_raw_iid(item), est_rating))

# Сортуємо рекомендації за прогнозованими оцінками
recommendations.sort(key=lambda x: x[1], reverse=True)

In [11]:
# 6. Виведи рекомендації (10 фільмів) для конкретного користувача.
top_10_recommendations = recommendations[:10]
print("Рекомендації для користувача:", user_id)
for movie_id, rating in top_10_recommendations:
    print(f"Фільм ID: {movie_id}, Прогнозована оцінка: {rating:.2f}")

Рекомендації для користувача: 186
Фільм ID: 300, Прогнозована оцінка: 4.82
Фільм ID: 318, Прогнозована оцінка: 4.71
Фільм ID: 488, Прогнозована оцінка: 4.63
Фільм ID: 89, Прогнозована оцінка: 4.61
Фільм ID: 513, Прогнозована оцінка: 4.57
Фільм ID: 79, Прогнозована оцінка: 4.55
Фільм ID: 1194, Прогнозована оцінка: 4.54
Фільм ID: 195, Прогнозована оцінка: 4.53
Фільм ID: 133, Прогнозована оцінка: 4.47
Фільм ID: 498, Прогнозована оцінка: 4.46


In [22]:
# Створюємо матрицю рейтингів (користувачі -> фільми)
ratings_matrix = df.pivot_table(index='user', columns='item', values='rating')

# Заповнюємо відсутні значення нулями (якщо користувач не оцінював фільм)
ratings_matrix = ratings_matrix.fillna(0)

In [23]:
# Функція для обчислення косинусної схожості
def cosine_similarity(u, v):
    return np.dot(u, v) / (norm(u) * norm(v))

# Створюємо матрицю схожості між користувачами
user_similarity_matrix = np.zeros((ratings_matrix.shape[0], ratings_matrix.shape[0]))

for i in range(ratings_matrix.shape[0]):
    for j in range(ratings_matrix.shape[0]):
        if i != j:
            user_similarity_matrix[i][j] = cosine_similarity(ratings_matrix.iloc[i], ratings_matrix.iloc[j])

In [24]:
# Прогнозуємо оцінку для фільму, котрий користувач ще не оцінював
def predict_rating(user, item, ratings_matrix, user_similarity_matrix):
    # Отримуємо індекси користувача
    user_index = user - 1  # Індекс в матриці починається з 0
    ratings = ratings_matrix[item]

    # Обчислюємо зважену середню оцінку
    weighted_sum = 0
    similarity_sum = 0

    for other_user_index in range(ratings_matrix.shape[0]):
        if other_user_index != user_index and ratings_matrix.iloc[other_user_index, item] != 0:
            similarity = user_similarity_matrix[user_index, other_user_index]
            weighted_sum += similarity * ratings_matrix.iloc[other_user_index, item]
            similarity_sum += similarity

    if similarity_sum == 0:
        return 0
    return weighted_sum / similarity_sum

In [29]:
def recommend_movies(user, ratings_matrix, user_similarity_matrix, num_recommendations=10):
    user_ratings = ratings_matrix.iloc[user_id - 1]
    recommendations = []

    for item in range(ratings_matrix.shape[1]):
        if user_ratings.iloc[item] == 0:  # Якщо користувач не оцінював цей фільм
            predicted_rating = predict_rating(user, item, ratings_matrix, user_similarity_matrix)
            recommendations.append((item, predicted_rating))

    # Сортуємо фільми за прогнозованими оцінками
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    # Повертаємо топ-10 рекомендацій
    return recommendations[:num_recommendations]


In [ ]:
def evaluate_own_model(ratings_matrix, user_similarity_matrix):
    actual_ratings = []
    predicted_ratings = []
    
    for user_id in range(1, ratings_matrix.shape[0] + 1):
        for item_id in range(ratings_matrix.shape[1]):
            actual_rating = ratings_matrix.iloc[user_id - 1, item_id]
            if actual_rating != 0:  # Ми можемо тільки оцінювати вже наявні рейтинги
                predicted_rating = predict_rating(user, item, ratings_matrix, user_similarity_matrix)
                actual_ratings.append(actual_rating)
                predicted_ratings.append(predicted_rating)

    mae = mean_absolute_error(actual_ratings, predicted_ratings)
    print(f"Середня абсолютна помилка (MAE) власної моделі: {mae:.4f}")
    return mae

# Оцінимо власну модель
own_model_mae = evaluate_own_model(ratings_matrix, user_similarity_matrix)

In [28]:
# Завантажуємо дані знову для бібліотеки Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user', 'item', 'rating']], reader)

# Розбиваємо на тренувальний і тестовий набори
trainset, testset = train_test_split(data, test_size=0.2)

# Використовуємо алгоритм SVD для побудови моделі
algo = SVD()
algo.fit(trainset)

# Прогнозуємо оцінки для тестового набору
predictions = algo.test(testset)

# Оцінимо точність моделі Surprise
surprise_mae = accuracy.mae(predictions)
print(f"Середня абсолютна помилка (MAE) моделі з Surprise: {surprise_mae:.4f}")

MAE:  0.7425
Середня абсолютна помилка (MAE) моделі з Surprise: 0.7425
